This script processes the data from the Instagram followers and following JSON files and creates dataframes to check the ppl that you follow but they don't follow you back and vice versa.

In [22]:
import json 
import pandas as pd

def parse_follow_data(folder_path, fol_type=['i_follow', 'follows_me']):
    """
    Parse followers/following json files
        :param folder_path: path to folder with json files; e.g. '/Users/rustya/Downloads/inst/connections/followers_and_following'
        :param fol_type: type of followers/following; e.g. ['i_follow', 'follows_me']
    returns: dataframe with followers/following
    """
    path = f'{folder_path}/followers_1.json' if fol_type == 'follows_me' else f'{folder_path}/following.json'

    with open(path) as f:
        data = json.load(f)
        data = data['relationships_following'] if fol_type == 'i_follow' else data

    d = {}
    df = pd.DataFrame(columns=['link', 'username', 'timestamp'])

    for row in data:
        d['link'] = row['string_list_data'][0]['href']
        d['username'] = row['string_list_data'][0]['value']
        d['timestamp'] = row['string_list_data'][0]['timestamp']
        df = pd.concat([df, pd.DataFrame(d, index=[0])], ignore_index=True)
        df[fol_type] = 1
    
    df[f'date_{fol_type}'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
    df = df.drop(columns=['timestamp']).sort_values([f'date_{fol_type}'], ascending=False)
    
    return df

In [23]:
fol_folder_path = 'your/path/here' # should end with /followers_and_following

In [24]:
following = parse_follow_data(fol_folder_path, fol_type='i_follow')
followers = parse_follow_data(fol_folder_path, fol_type='follows_me')

joined = pd.merge(following, followers, on=['username', 'link'], how='outer').fillna(0)

In [ ]:
follows_me_but_i_dont = joined[(joined['i_follow'] == 0) & (joined['follows_me'] == 1)]
print(f"Num of followers that I don't follow back: {len(follows_me_but_i_dont)}")
follows_me_but_i_dont

In [ ]:
i_follow_but_they_dont = joined[(joined['i_follow'] == 1) & (joined['follows_me'] == 0)]
print(f"Num of followers that don't follow me back: {len(i_follow_but_they_dont)}")
i_follow_but_they_dont